In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats
from ta.momentum import RSIIndicator

In [2]:
from ta.momentum import StochasticOscillator
def Stochastic(window_para,smwd_para, highband, to_csv):
    lowband = 100 - highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        high_weekly = result[result['code']==code]['high_weekly']
        low_weekly = result[result['code']==code]['low_weekly']
        result.loc[code == result['code'].shift(window_para),'stoch'] = StochasticOscillator(high = high_weekly, low = low_weekly, close=close_weekly, window=window_para, smooth_window = smwd_para).stoch()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['stoch']<lowband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['stoch']>highband),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband

In [3]:
optimize_result = []
for window_para in range(2,30, 2):
    for smwd_para in range(10,50,10):
        for highband in range(50,100, 5):
            x=Stochastic(window_para,smwd_para/10, highband, to_csv = False)
            optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.17853778181134738
MDD 0.1822733030536781
MDD 0.18558068512870501
MDD 0.19406669267585439
MDD 0.1996600249333875
MDD 0.19805798017777632
MDD 0.19722069936214978
MDD 0.19510502144474606
MDD 0.19986709308147116
MDD 0.20767975414890963
MDD 0.17853778181134738
MDD 0.1822733030536781
MDD 0.18558068512870501
MDD 0.19406669267585439
MDD 0.1996600249333875
MDD 0.19805798017777632
MDD 0.19722069936214978
MDD 0.19510502144474606
MDD 0.19986709308147116
MDD 0.20767975414890963
MDD 0.17853778181134738
MDD 0.1822733030536781
MDD 0.18558068512870501
MDD 0.19406669267585439
MDD 0.1996600249333875
MDD 0.19805798017777632
MDD 0.19722069936214978
MDD 0.19510502144474606
MDD 0.19986709308147116
MDD 0.20767975414890963
MDD 0.17853778181134738
MDD 0.1822733030536781
MDD 0.18558068512870501
MDD 0.19406669267585439
MDD 0.1996600249333875
MDD 0.19805798017777632
MDD 0.19722069936214978
MDD 0.19510502144474606
MDD 0.19986709308147116
MDD 0.20767975414890963
MDD 0.18150370365344184
MDD 0.18598340191235735


MDD 0.2131448790397369
MDD 0.226157760962908
MDD 0.24373841744482935
MDD 0.25252570179064593
MDD 0.19483264138608017
MDD 0.19734073975155425
MDD 0.20444642525547413
MDD 0.20759364657209994
MDD 0.21105782870708903
MDD 0.20984017436216798
MDD 0.2131448790397369
MDD 0.226157760962908
MDD 0.24373841744482935
MDD 0.25252570179064593
MDD 0.1946612126751509
MDD 0.19825198369189603
MDD 0.20267668427722224
MDD 0.20894835653485402
MDD 0.21108598482707802
MDD 0.2200937463904871
MDD 0.22494415643475563
MDD 0.24309540153952586
MDD 0.26145442525343376
MDD 0.2623155062162782
MDD 0.1946612126751509
MDD 0.19825198369189603
MDD 0.20267668427722224
MDD 0.20894835653485402
MDD 0.21108598482707802
MDD 0.2200937463904871
MDD 0.22494415643475563
MDD 0.24309540153952586
MDD 0.26145442525343376
MDD 0.2623155062162782
MDD 0.1946612126751509
MDD 0.19825198369189603
MDD 0.20267668427722224
MDD 0.20894835653485402
MDD 0.21108598482707802
MDD 0.2200937463904871
MDD 0.22494415643475563
MDD 0.24309540153952586
MDD 0.

In [5]:
Stochastic(10,3, 55, to_csv = False)

MDD 0.19623493789406737


(4.558353678259922, 10, 55)

In [4]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2
161,4.558354,10,55
181,4.558354,10,55
191,4.558354,10,55
171,4.558354,10,55
17,4.552051,2,85
7,4.552051,2,85
27,4.552051,2,85
37,4.552051,2,85
28,4.428954,2,90
18,4.428954,2,90
